# Speech Recognition

In [5]:
! pip install torchaudio
! pip install datasets
! pip install sox

In [10]:
import numpy as np
import zipfile
import gc
import cv2
import math
import warnings
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import torchvision
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.utils.data as data_utils
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
# import torchmetrics
import torch.nn.functional as F
import PIL
import torch.utils.data as data_utils
import json
import sox
import struct

# import pytorch_lightning as pl
import imutils
import zipfile

from torch.utils.data import Dataset
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python.client import device_lib
from zipfile import ZipFile
from IPython import display
from torchvision import models, transforms
from google.colab.patches import cv2_imshow
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.preprocessing import LabelEncoder
from torchvision.models.feature_extraction import create_feature_extractor
from datasets import load_dataset
from PIL import Image
from collections import defaultdict
from google.colab import drive

Get the dataset

In [3]:
! wget https://www.openslr.org/resources/12/train-clean-100.tar.gz -O libspeech.tar.gz
! tar -xzf libspeech.tar.gz

--2023-11-12 15:04:53--  https://www.openslr.org/resources/12/train-clean-100.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/12/train-clean-100.tar.gz [following]
--2023-11-12 15:04:54--  http://us.openslr.org/resources/12/train-clean-100.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘libspeech.tar.gz’

libspeech.tar.gz    100%[===================>]   5.95G  35.2MB/s    in 2m 57s  

2023-11-12 15:07:51 (34.4 MB/s) - ‘libspeech.tar.gz’ saved [6387309499/6387309499]



In [23]:
def open_file(path):
  return open(path, mode = "rb")

## Data Analysis

Note that LibSpeech dataset has a flac format!

### Basic Audio File Format Analysis

In [28]:
def get_file_type(binary_data):
  file_type = binary_data.read(4)
  return file_type

def get_metadata_block_header(binary_data):
  METADATA_BLOCK_TYPE = {
    0: "STREAMINFO",
    1: "PADDING",
    2: "APPLICATION",
    3: "SEEKTABLE",
    4: "VORBIS_COMMENT",
    5: "CUESHEET",
    6: "PICTURE",
  }

  metadata_block_header_packed = binary_data.read(4)

  # Unpack the bytes as an unsigned integer
  metadata_block_header_unpacked = struct.unpack('>I', metadata_block_header_packed)[0]

  metadata_block_header_unpacked_copied = metadata_block_header_unpacked

  # Extract and print 1 bit
  sample_rate = (metadata_block_header_unpacked_copied >> 31) & 0x1

  # Extract and print the next 7 bits
  block_type = (metadata_block_header_unpacked_copied >> 24) & 0b1111111
  block_type = METADATA_BLOCK_TYPE[block_type]

  # Extract and print the next 24 bits
  length_of_metadata = metadata_block_header_unpacked_copied & 0xFFFFFF
  return sample_rate, block_type, length_of_metadata

def get_metadata_block_streaminfo(binary_data):
  """
  Mandatory information for FLAC.
  """

  minblocksize_packed = binary_data.read(2)
  maxblocksize_framesize_packed = binary_data.read(8)
  samplerate_nochannels_bits_totalsample_packed = binary_data.read(8)

  # Unpack the data using struct
  minblocksize_unpacked = struct.unpack(
      ">H", minblocksize_packed
  )[0]
  maxblocksize_framesize_unpacked = struct.unpack(
      ">Q", maxblocksize_framesize_packed
  )[0]
  samplerate_nochannels_bits_totalsample_unpacked = struct.unpack(
      ">Q", samplerate_nochannels_bits_totalsample_packed
  )[0]

  # Extract and print the specified bits
  minblocksize_packed_unpacked_copied = minblocksize_unpacked
  maxblocksize_framesize_unpacked_copied = maxblocksize_framesize_unpacked
  samplerate_nochannels_bits_totalsample_unpacked_copied = samplerate_nochannels_bits_totalsample_unpacked

  min_block_size = minblocksize_packed_unpacked_copied

  # Extract and print the first 16 bits
  max_block_size = (maxblocksize_framesize_unpacked_copied >> 48) & 0xFFFF

  # Extract and print the next 24 bits
  min_frame_size = (maxblocksize_framesize_unpacked_copied >> 24) & 0xFFFFFF

  # Extract and print the next 24 bits
  max_frame_size = maxblocksize_framesize_unpacked_copied & 0xFFFFFF

  # Extract and print the first 20 bits
  sample_rate = (samplerate_nochannels_bits_totalsample_unpacked_copied >> 44) & 0xFFFFF

  # Extract and print the next 3 bits
  no_channels = (samplerate_nochannels_bits_totalsample_unpacked_copied >> 41) & 0b111
  no_channels = no_channels + 1

  # Extract and print the next 5 bits
  bits = (samplerate_nochannels_bits_totalsample_unpacked_copied >> 36) & 0b11111
  bits = bits + 1

  # Extract and print the next 36 bits
  total_sample = samplerate_nochannels_bits_totalsample_unpacked_copied & 0xFFFFFFFFF

  return min_block_size, max_block_size, sample_rate, no_channels, bits, total_sample

In [29]:
def get_audio_format_data(audio_data):
  file_type = get_file_type(audio_data)
  sample_rate, block_type, length_of_metadata = get_metadata_block_header(
      audio_data
  )
  min_block_size, max_block_size, sample_rate, no_channels, bits, total_sample = get_metadata_block_streaminfo(
      audio_data
  )

  return {
      "file_type": file_type,
      "sample_rate": sample_rate,
      "block_type": block_type,
      "length_of_metadata": length_of_metadata,
      "min_block_size": min_block_size,
      "max_block_size": max_block_size,
      "sample_rate": sample_rate,
      "no_channels": no_channels,
      "bits": bits,
      "total_sample": total_sample,
  }

sample_audio_data = open_file("/content/LibriSpeech/train-clean-100/103/1240/103-1240-0000.flac")
get_audio_format_data(sample_audio_data)

{'file_type': b'fLaC',
 'sample_rate': 16000,
 'block_type': 'STREAMINFO',
 'length_of_metadata': 34,
 'min_block_size': 4096,
 'max_block_size': 4096,
 'no_channels': 1,
 'bits': 16,
 'total_sample': 225360}

### Spectrum and Waveform Analysis